### Step 0
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

In [38]:
import pandas as pd
energy = pd.read_excel('data/Energy Indicators.xls',
                            usecols= range(2,6),
                            nrows = 227,                        
                            skiprows = range(1,16),header=1,na_values=['...'])
energy = pd.DataFrame(energy)
energy = energy.rename(columns={ "Unnamed: 2": "Country","Renewable Electricity Production": "% Renewable"})
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x*1000000)
energy.at[0,'Energy Supply']= 'Gigajoules'
energy["Country"].fillna("np.NaN", inplace = True)

def CountryStrOnly(text):
    onlyStr = ''.join([item for item in text if not item.isdigit()])
    return onlyStr

energy['Country'] = energy['Country'].apply(CountryStrOnly)

energy['Country'] = energy['Country'].str.replace(r" \(.*\)","")

energy['Country'] = energy['Country'].replace({ "Republic of Korea": "South Korea" ,
                                                "United States of America": "United States",
                                                "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                                "China, Hong Kong Special Administrative Region": "Hong Kong"})

energy.head()

,Country,Energy Supply,Energy Supply per capita,% Renewable
0,np.NaN,Gigajoules,Gigajoules,%
1,Afghanistan,321000000,10,78.6693
2,Albania,102000000,35,100
3,Algeria,1959000000,51,0.55101
4,American Samoa,NaN,NaN,0.641026


### Step 1
<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

In [11]:
import pandas as pd
GDP = pd.read_csv('data/world_bank.csv', header=1 ,skiprows = range(1,4))
GDP.replace("Korea, Rep.", "South Korea",inplace = True)
GDP.replace("Iran, Islamic Rep.","Iran",inplace = True)
GDP.replace("Hong Kong SAR, China","Hong Kong",inplace = True)
GDP.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10


### Step 2
Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

In [12]:
ScimEn =pd.read_excel('data/scimagojr-3.xlsx')
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


### Step 3
Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This step should yeild a DataFrame with 20 columns and 15 entries.*

In [35]:
GDP = GDP.rename(columns={ "Country Name": "Country"})
GDP_energy= pd.merge(GDP[['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']],
                      ScimEn[ (ScimEn['Rank'] <= 15 )],
                      on='Country')
join_result = pd.merge(GDP_energy, 
                      energy,
                      on='Country')
join=join_result.set_index('Country')
join=join.sort_values(by=['Rank'])
join.head(15)


,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable
Country,,,,,,,,,,,,,,,,,,,,
China,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,1,127050,126767,597237,411683,4.70,138,127191000000,93,19.7549
United States,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,2,96661,94747,792274,265436,8.20,230,90838000000,286,11.571
Japan,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,3,30504,30287,223024,61554,7.31,134,18984000000,149,10.2328
United Kingdom,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,4,20944,20357,206091,37874,9.84,139,7920000000,124,10.6005
Russian Federation,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,5,18534,18301,34266,12422,1.85,57,30709000000,214,17.2887
Canada,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,6,17899,17620,215003,40930,12.01,149,10431000000,296,61.9454
Germany,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,7,17027,16831,140566,27426,8.26,126,13261000000,165,17.9015
India,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,8,15005,14841,128763,37209,8.58,115,33195000000,26,14.9691
France,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,9,13153,12973,130632,28601,9.93,114,10597000000,166,17.0203


### Step 4
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This step should yield a single number.*

In [14]:
GDP = GDP.rename(columns={ "Country Name": "Country"})
GDP_energy= pd.merge(GDP[['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']],
                      ScimEn[ (ScimEn['Rank'] <= 15 )],
                      on='Country' , how='left')
join_result = pd.merge(GDP_energy, 
                      energy,
                      on='Country', how='left')
join=join_result.set_index('Country')
join=join.sort_values(by=['Rank'])
join.shape

(264, 20)

In [ ]:
249

### Step 5

#### Answer the following questions in the context of only the top 15 countries by Scimagojr Rank 


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This step should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [19]:
join['mean']=join[['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']].mean(axis=1,skipna=True)
join = join.sort_values(by=['mean'],ascending=False)
join['mean']

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: mean, dtype: float64

### Step  6
What is the mean `Energy Supply per Capita`?

*This step should return a single number.*

In [20]:
Energy_Supply_per_capita=join['Energy Supply per capita']
Mean_Value=Energy_Supply_per_capita.mean()
Mean_Value

157.6

### Step 7
What country has the maximum % Renewable and what is the percentage?

*This step should return a tuple with the name of the country and the percentage.*

In [34]:
df7 = join.copy()

max = join['% Renewable'].max()
df7 = join[ (join['% Renewable'] == max ) ]
df7=df7.reset_index()

subset = df7[['Country','% Renewable']]
tuples = [tuple(x) for x in subset.values][0]
tuples


('Brazil', 69.64803)

### Step 8
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This step should return a tuple with the name of the country and the ratio.*

In [8]:
df8 = join.copy()

df8=df8.reset_index()

df8['Ratio'] = df8['Self-citations']/df8['Citations'].sum()
max = df8['Ratio'].max()

max_ratio = df8[ (df8['Ratio'] == max ) ]
subset_ratio = max_ratio[['Country','Ratio']]
tuple_ratio = [tuple(x) for x in subset_ratio.values][0]
tuple_ratio


('China', 0.1360191815780727)

### Step 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This step should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [9]:
DF9 = join.copy()
Renewable=join['% Renewable']
Mean_Value=Renewable.median()
DF9['HighRenew'] = join['% Renewable'].apply(lambda x : '1' if x >= Mean_Value else '0' )
#type(DF9['HighRenew'])
DF9['HighRenew']

Country
United States         0
China                 1
Japan                 0
Germany               1
France                1
United Kingdom        0
Brazil                1
Italy                 1
India                 0
Canada                1
Russian Federation    1
Spain                 1
Australia             0
South Korea           0
Iran                  0
Name: HighRenew, dtype: object

### Step 10
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [73]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
DF_Dict = pd.DataFrame(ContinentDict.items(), columns=['Country', 'Continent'])

join['Population'] = join['Energy Supply']/join['Energy Supply per capita']

DF = pd.merge(DF_Dict,
              join[['Population']],
              on='Country')
DF


,Country,Continent,Population
0,China,Asia,1.36765e+09
1,United States,North America,3.17615e+08
2,Japan,Asia,1.27409e+08
3,United Kingdom,Europe,6.3871e+07
4,Russian Federation,Europe,1.435e+08
5,Canada,North America,3.52399e+07
6,Germany,Europe,8.03697e+07
7,India,Asia,1.27673e+09
8,France,Europe,6.38373e+07
9,South Korea,Asia,4.98054e+07


In [84]:

DF['Population'] = DF.Population.astype(float)

DF_Final = DF.groupby('Continent')['Population'].agg(['sum','count','std','mean'])
DF_Final 

,sum,count,std,mean
Continent,,,,
Asia,2.898666e+09,5,6.790979e+08,5.797333e+08
Australia,2.331602e+07,1,NaN,2.331602e+07
Europe,4.579297e+08,6,3.464767e+07,7.632161e+07
North America,3.528552e+08,2,1.996696e+08,1.764276e+08
South America,2.059153e+08,1,NaN,2.059153e+08
